The goal of this notebook is to tie the songs I know into the era that they are from. 

For now I am just focusing on non classical music, essentially 20th and 21st century.

In [9]:
# Refactored pipeline: artists by decade (clean copy-ready Markdown output)
# This cell is self-contained and does not rely on earlier cells.

import pandas as pd
import numpy as np
from IPython.display import Markdown, display
from pathlib import Path

# ----------------------
# Helpers
# ----------------------
# Filters to exclude "classical" content
FILTER_GENRE_TERMS = ['classical', 'choral', 'baroque']
FILTER_ALBUM_TERMS = ['Bach']


def clean_artist_col(series: pd.Series) -> pd.Series:
    """Return primary artist (first before comma), trimmed."""
    return series.astype(str).str.split(',').str[0].str.strip()


def infer_year_from(
    df: pd.DataFrame,
    candidates=('Album Date', 'Release Date'),
    text_cols=('Album', 'Song', 'Title', 'Track', 'Name'),
) -> pd.Series:
    """Infer a year from date columns first, then regex from text columns."""
    year = pd.Series(np.nan, index=df.index, dtype='float')

    for col in candidates:
        if col in df.columns:
            # Try parsing as datetime then take year
            y_dt = pd.to_datetime(df[col], errors='coerce', utc=True).dt.year
            year = year.fillna(y_dt)

            # Try numeric-as-year if provided directly
            y_num = pd.to_numeric(df[col], errors='coerce')
            y_num = y_num.where(y_num.between(1800, 2099))
            year = year.fillna(y_num)

    # Fallback regex search for a 4-digit plausible year in text fields
    regex = r'(?<!\d)(18|19|20)\d{2}(?!\d)'
    for col in text_cols:
        if col in df.columns:
            m = df[col].astype(str).str.extract(regex, expand=False)
            y = pd.to_numeric(m, errors='coerce')
            year = year.fillna(y)

    return year.astype('Int64')


def to_decade(year: pd.Series) -> pd.Series:
    return ((year // 10) * 10).astype('Int64')


def ascii_hist(series: pd.Series, width: int = 50, bar_char: str = '█') -> str:
    if series.empty:
        return '(no data)'
    values = series.sort_index()
    maxv = values.max()
    lines = []
    for idx, val in values.items():
        bar_len = int(round((val / maxv) * width)) if maxv else 0
        lines.append(f"{int(idx)}s | {bar_char * bar_len} {val}")
    return "\n".join(lines)


# ----------------------
# Load data & apply filters
# ----------------------
all_path = 'all.csv'
df_all = pd.read_csv(all_path)

# Build exclusion masks (case-insensitive)
genres = df_all.get('Genres', pd.Series('', index=df_all.index)).astype(str)
albums = df_all.get('Album', pd.Series('', index=df_all.index)).astype(str)

# If 'pop' appears in Genres, do not exclude by classical-like terms
allowed_pop = genres.str.contains('pop', case=False, na=False)
genre_mask_raw = genres.str_contains('|'.join(FILTER_GENRE_TERMS), case=False, na=False) if hasattr(genres.str, 'str_contains') else genres.str.contains('|'.join(FILTER_GENRE_TERMS), case=False, na=False)
genre_mask = genre_mask_raw & ~allowed_pop

album_mask = albums.str.contains('|'.join(FILTER_ALBUM_TERMS), case=False, na=False)

excluded_mask = genre_mask | album_mask

df_excluded = df_all[excluded_mask].copy()
df_included = df_all[~excluded_mask].copy()

# Clean artists (both included and excluded)
for frame in (df_included, df_excluded):
    frame['Cleaned_Artist'] = clean_artist_col(frame.get('Artist', pd.Series('', index=frame.index)))

# ----------------------
# Derive Year/Decade and aggregate
# ----------------------
df_included['Year'] = infer_year_from(df_included)
df_included['Decade'] = to_decade(df_included['Year'])

# Count artists per decade, after filtering to artists with >1 tracks in that decade
raw = df_included.dropna(subset=['Decade']).groupby(['Decade', 'Cleaned_Artist']).size()
filtered = raw[raw > 1]
artist_decade_counts = filtered.groupby(level=0).size().sort_index()
by_decade = filtered.sort_values(ascending=False).groupby(level=0).head(100)

# ----------------------
# Markdown export (copy-ready)
# ----------------------
lines = []
lines.append('# Music decades report')
lines.append('')

included_tracks = len(df_included)
unique_artists = df_included['Cleaned_Artist'].nunique()
with_year = int(df_included['Year'].notna().sum())
missing_year = int(df_included['Year'].isna().sum())

lines.append('Summary:')
lines.append(f'- Included tracks: {included_tracks:,} (unique artists: {unique_artists:,})')
lines.append(f'- Year coverage: {with_year:,} with year, {missing_year:,} missing')
lines.append('')

lines.append('## Artists by decade (count of artists with >1 tracks, per decade)')
if artist_decade_counts.empty:
    lines.append('_No qualifying artists_')
else:
    lines.append(artist_decade_counts.to_frame('Artists').to_markdown())
lines.append('')

lines.append('### ASCII histogram')
lines.append('')
lines.append('```')
lines.append(ascii_hist(artist_decade_counts))
lines.append('```')
lines.append('')

lines.append('## Top artists per decade (up to 100 each)')
if by_decade.empty:
    lines.append('_No data_')
else:
    for dec in sorted(by_decade.index.get_level_values(0).unique()):
        sub = by_decade.xs(dec, level=0)
        lines.append(f'### {int(dec)}s')
        lines.extend([f'- {artist}: {cnt}' for artist, cnt in sub.items()])
        lines.append('')

# Supplemental: Missing-year artists (within included set)
lines.append('## Supplemental: Artists with tracks excluded due to missing Year')
missing_by_artist = (
    df_included[df_included['Year'].isna()]
    .groupby('Cleaned_Artist')
    .size()
    .sort_values(ascending=False)
)
if missing_by_artist.empty:
    lines.append('_None_')
else:
    total_missing_tracks = int(df_included['Year'].isna().sum())
    lines.append(f'- Total missing-year tracks: {total_missing_tracks:,}; affected artists: {missing_by_artist.shape[0]:,}')
    top_missing = missing_by_artist.head(50)
    lines.append('')
    lines.append('| Artist | Missing tracks |')
    lines.append('| --- | ---: |')
    for artist, cnt in top_missing.items():
        lines.append(f'| {artist} | {int(cnt)} |')
    rem = int(missing_by_artist.shape[0] - top_missing.shape[0])
    if rem > 0:
        lines.append(f'_…and {rem} more_')
lines.append('')

# Supplemental: Excluded-by-filters artists (classical/choral/baroque/Bach)
lines.append('## Supplemental: Artists excluded by classical filters')
excluded_counts = df_excluded.groupby('Cleaned_Artist').size().sort_values(ascending=False)
lines.append(f'- Excluded tracks: {len(df_excluded):,}; unique excluded artists: {excluded_counts.index.nunique():,}')
if excluded_counts.empty:
    lines.append('_None_')
else:
    TOP_N_EXCL = 50
    top_excl = excluded_counts.head(TOP_N_EXCL)
    lines.append('')
    lines.append('| Artist | Tracks Excluded |')
    lines.append('| --- | ---: |')
    for artist, cnt in top_excl.items():
        lines.append(f'| {artist} | {int(cnt)} |')
    rem = int(excluded_counts.shape[0] - top_excl.shape[0])
    if rem > 0:
        lines.append(f'_…and {rem} more_')

# Cross-check: artists appearing in both included and excluded sets
included_counts = df_included.groupby('Cleaned_Artist').size().sort_values(ascending=False)
common = excluded_counts.index.intersection(included_counts.index)
if len(common) > 0:
    lines.append('')
    lines.append('## Cross-check: Excluded artists also present in included set')
    overlap = (
        pd.DataFrame({
            'Excluded': excluded_counts.loc[common],
            'Included': included_counts.loc[common],
        })
        .sort_values(['Excluded', 'Included'], ascending=[False, False])
    )
    top_overlap = overlap.head(25)
    lines.append('| Artist | Excluded | Included |')
    lines.append('| --- | ---: | ---: |')
    for artist, row in top_overlap.iterrows():
        lines.append(f'| {artist} | {int(row.Excluded)} | {int(row.Included)} |')

# Emit Markdown (display + write to file)
md_report = '\n'.join(lines)
display(Markdown(md_report))

# Write to a standalone file for easy viewing
out_path = Path('music_decades_report.md')
out_path.write_text(md_report, encoding='utf-8')
print(f"\nSaved Markdown to: {out_path.resolve()}\n")
print(md_report)

# Music decades report

Summary:
- Included tracks: 1,264 (unique artists: 430)
- Year coverage: 1,120 with year, 144 missing

## Artists by decade (count of artists with >1 tracks, per decade)
|   Decade |   Artists |
|---------:|----------:|
|     1950 |         7 |
|     1960 |        10 |
|     1970 |        19 |
|     1980 |         6 |
|     1990 |        21 |
|     2000 |        42 |
|     2010 |        33 |
|     2020 |         8 |

### ASCII histogram

```
1950s | ████████ 7
1960s | ████████████ 10
1970s | ███████████████████████ 19
1980s | ███████ 6
1990s | █████████████████████████ 21
2000s | ██████████████████████████████████████████████████ 42
2010s | ███████████████████████████████████████ 33
2020s | ██████████ 8
```

## Top artists per decade (up to 100 each)
### 1950s
- Ella Fitzgerald: 5
- Julie London: 4
- The Four Freshmen: 3
- The Chordettes: 3
- Nat King Cole: 2
- Miles Davis: 2
- Charlie Parker: 2

### 1960s
- The Beatles: 30
- The Beach Boys: 20
- Stan Getz: 5
- The Doors: 4
- Bob Dylan: 3
- Simon & Garfunkel: 2
- Nat King Cole: 2
- Buffalo Springfield: 2
- Louis Armstrong: 2
- The Monkees: 2

### 1970s
- Steely Dan: 9
- Elton John: 7
- Queen: 5
- Billy Joel: 5
- Stevie Wonder: 4
- Electric Light Orchestra: 4
- Earth: 3
- Bruce Springsteen: 3
- Bee Gees: 2
- The Beach Boys: 2
- Supertramp: 2
- Steve Miller Band: 2
- Squeeze: 2
- Neil Young: 2
- Bob Dylan: 2
- Dire Straits: 2
- Eagles: 2
- The Hollies: 2
- The Beatles: 2

### 1980s
- Pixies: 6
- Tears For Fears: 5
- Billy Joel: 3
- Erasure: 2
- Michael Jackson: 2
- Duran Duran: 2

### 1990s
- Elliott Smith: 29
- Green Day: 19
- Buena Vista Social Club: 14
- Weezer: 12
- Radiohead: 8
- Nirvana: 5
- The Notorious B.I.G.: 4
- Nas: 4
- Ms. Lauryn Hill: 4
- The Cranberries: 3
- Big L: 3
- Benny Golson: 3
- Cocteau Twins: 3
- Smash Mouth: 2
- Jeff Buckley: 2
- Sugar Ray: 2
- Norbert Kraft: 2
- Heatmiser: 2
- Adam Holzman: 2
- A Tribe Called Quest: 2
- Depeche Mode: 2

### 2000s
- The Shins: 51
- Elliott Smith: 47
- of Montreal: 42
- The Strokes: 30
- Radiohead: 23
- Franz Ferdinand: 22
- Green Day: 9
- Hot Hot Heat: 9
- Muse: 9
- Fountains Of Wayne: 9
- Weezer: 6
- Antonio Lauro: 5
- Modest Mouse: 4
- Wilco: 4
- Interpol: 4
- The White Stripes: 4
- MGMT: 3
- Simple Plan: 3
- Emitt Rhodes: 3
- Good Charlotte: 3
- Elefant: 3
- The All-American Rejects: 3
- Vampire Weekend: 3
- The Fratellis: 3
- Arctic Monkeys: 3
- Streetlight Manifesto: 3
- Andrew Bird: 2
- Blonde Redhead: 2
- Bobby Darin: 2
- Beyoncé: 2
- Avril Lavigne: 2
- Britney Spears: 2
- DANGERDOOM: 2
- Albert Hammond Jr: 2
- Pendulum: 2
- OK Go: 2
- Lily Allen: 2
- Matchbox Twenty: 2
- John Adams: 2
- Talbot Tagora: 2
- The Bravery: 2
- The New Pornographers: 2

### 2010s
- Alvvays: 18
- Pile: 15
- Ariana Grande: 13
- Naomi Punk: 12
- MGMT: 12
- Kendrick Lamar: 7
- Crumb: 5
- Mac DeMarco: 4
- Parquet Courts: 4
- James Elkington: 4
- Two Door Cinema Club: 4
- of Montreal: 3
- Ty Segall: 3
- Foster The People: 3
- Dirty Projectors: 3
- Caroline Shaw: 3
- Deca: 3
- Broken Bells: 2
- Big Thief: 2
- Peebs The Prophet: 2
- Nathan Salsburg: 2
- Open Mike Eagle: 2
- Jaeden Camstra: 2
- Girls: 2
- Katy Perry: 2
- Jonwayne: 2
- Freddie Gibbs: 2
- Passion Pit: 2
- Tyler: 2
- The Strokes: 2
- Sky Ferreira: 2
- Steve Lacy: 2
- Tame Impala: 2

### 2020s
- SPIRIT OF THE BEEHIVE: 3
- Brendan Eder Ensemble: 3
- Nicholas Britell: 3
- Sabrina Carpenter: 3
- Taylor Swift: 3
- Adrianne Lenker: 3
- Beyoncé: 3
- Alex G: 2

## Supplemental: Artists with tracks excluded due to missing Year
- Total missing-year tracks: 144; affected artists: 78

| Artist | Missing tracks |
| --- | ---: |
| of Montreal | 15 |
| Bill Evans | 14 |
| Os Tincoãs | 12 |
| Kendrick Lamar | 7 |
| Muse | 4 |
| ABBA | 4 |
| Billy Joel | 3 |
| The Turtles | 3 |
| Electric Light Orchestra | 3 |
| Stan Getz | 3 |
| Bobby Darin | 2 |
| BOYS LIKE GIRLS | 2 |
| Barry Manilow | 2 |
| America | 2 |
| Masayoshi Takanaka | 2 |
| Katy Perry | 2 |
| The Millennium | 2 |
| Madness | 2 |
| Blue Scholars | 1 |
| Andrew Gold | 1 |
| Antônio Carlos Jobim | 1 |
| Blue Öyster Cult | 1 |
| Carole King | 1 |
| Buffalo Springfield | 1 |
| Blur | 1 |
| Donovan | 1 |
| Don McLean | 1 |
| Charles Mingus | 1 |
| Chet Baker | 1 |
| Cheap Trick | 1 |
| Glenn Miller | 1 |
| Gerry Rafferty | 1 |
| Fred Astaire | 1 |
| Frank Sinatra | 1 |
| Gorillaz | 1 |
| Harry Nilsson | 1 |
| Heart | 1 |
| Imogen Heap | 1 |
| John Denver | 1 |
| John Coltrane | 1 |
| Earth | 1 |
| Five For Fighting | 1 |
| DEVO | 1 |
| Daryl Hall & John Oates | 1 |
| Dean Martin | 1 |
| Count Bass D | 1 |
| Modest Mouse | 1 |
| Looking Glass | 1 |
| Julie London | 1 |
| Judee Sill | 1 |
_…and 28 more_

## Supplemental: Artists excluded by classical filters
- Excluded tracks: 959; unique excluded artists: 190

| Artist | Tracks Excluded |
| --- | ---: |
| Johann Sebastian Bach | 88 |
| Philip Glass | 45 |
| Frédéric Chopin | 37 |
| Göran Söllscher | 34 |
| Catrin Finch | 32 |
| András Schiff | 31 |
| Wolfgang Amadeus Mozart | 29 |
| Julian Rachlin | 27 |
| Augsburger Domsingknaben | 24 |
| Emerson String Quartet | 21 |
| Maurice Ravel | 20 |
| Eduardo Fernandez | 19 |
| Ludwig van Beethoven | 17 |
| Vladimir Ashkenazy | 17 |
| Víkingur Ólafsson | 15 |
| Rosalyn Tureck | 13 |
| Andrés Segovia | 13 |
| Arthur Grumiaux | 13 |
| Nicanor Zabaleta | 12 |
| Claude Debussy | 12 |
| Christiane Oelze | 11 |
| Mischa Maisky | 11 |
| Jean Louis Steuerman | 10 |
| Andrei Gavrilov | 10 |
| Janine Jansen | 9 |
| Manuel Seco de Arpe | 9 |
| Ramón Paus | 9 |
| David Lang | 8 |
| The Sarum Consort | 8 |
| Choir of Gonville & Caius College | 8 |
| George Frideric Handel | 8 |
| Henry Purcell | 8 |
| Claudio Arrau | 7 |
| Joseph Haydn | 7 |
| Severino Gazzelloni | 6 |
| Robert Schumann | 6 |
| Pepe Romero | 6 |
| Wilhelm Kempff | 6 |
| Jian Wang | 6 |
| Irena Grafenauer | 6 |
| Gabriel Fauré | 6 |
| Franz Schubert | 6 |
| Heinz Holliger | 5 |
| Giuliano Carmignola | 5 |
| John C. Williams | 5 |
| Alexandre Lagoya | 5 |
| Wolfgang Rübsam | 5 |
| Antonio Vivaldi | 4 |
| Amsterdam Loeki Stardust Quartet | 4 |
| Arcangelo Corelli | 4 |
_…and 140 more_

## Cross-check: Excluded artists also present in included set
| Artist | Excluded | Included |
| --- | ---: | ---: |
| Catrin Finch | 32 | 1 |
| John Adams | 2 | 2 |


Saved Markdown to: C:\Users\jacks\code\music\music_decades_report.md

# Music decades report

Summary:
- Included tracks: 1,264 (unique artists: 430)
- Year coverage: 1,120 with year, 144 missing

## Artists by decade (count of artists with >1 tracks, per decade)
|   Decade |   Artists |
|---------:|----------:|
|     1950 |         7 |
|     1960 |        10 |
|     1970 |        19 |
|     1980 |         6 |
|     1990 |        21 |
|     2000 |        42 |
|     2010 |        33 |
|     2020 |         8 |

### ASCII histogram

```
1950s | ████████ 7
1960s | ████████████ 10
1970s | ███████████████████████ 19
1980s | ███████ 6
1990s | █████████████████████████ 21
2000s | ██████████████████████████████████████████████████ 42
2010s | ███████████████████████████████████████ 33
2020s | ██████████ 8
```

## Top artists per decade (up to 100 each)
### 1950s
- Ella Fitzgerald: 5
- Julie London: 4
- The Four Freshmen: 3
- The Chordettes: 3
- Nat King Cole: 2
- Miles Davis: 2
- Charlie Parker